# Holiday Manager

## Pseudocode
We are looking to create an application which manages holidays. Before we begin with the application, we will pre-load the holidays from https://www.timeanddate.com/holidays/us/2022?hol=33554809 into a JSON file along with the pre-loaded holidays already loaded in the JSON file.

With the holidays approaching, the marketing person may want to know what the weather looks like so they can determine what to market more. Use a weather API (e.g. [Open Weather Map API](https://rapidapi.com/community/api/open-weather-map)) to show what the weather looks like for the current period (more in the UI explanation below).

**Start Up**
* Display welcome message/UI info
* Display number of holidays stored in the system (e.g. JSON file)

**Main Menu**
* Display message Holiday menu
* Allow user to choose from:
> 1. Add a Holiday
> 2. Remove a Holiday
> 3. Save Holiday List
> 4. View Holidays
> 5. Exit

**Add a Holiday**
* Display Add a Holiday
* Prompt the user for Holiday name and Date
> * If date is typed incorrectly (i.e. doesn't fit into ISO 8601 YYYY-MM-DD), display error message 'Invalid date' and reprompt
> * If date is typed correctly, display success message '... has been added to list' and add holiday to list

**Remove a Holiday**
* Display Remove a Holiday
* Prompt user for Holiday Name
> * If user inputs a holiday not in file, display error message 'HolidayName not found' and reprompt
> * If user inputs holiday in file, display success message and remove from holiday list

**Save Holiday List**
* Display Save Holiday List
* Prompt user if they want to save their changes
> * If no, do not save and display canceled message 'Holiday list file save cancelled'
> * If yes, save holiday list to JSON and display success message 'Changes saved'

**View Holidays**
* Display View Holidays
* Prompt user for year
* Prompt user for week#, leave blank if current week
> * if week# blank, prompt user if they would like to see this week's weather
> * Display holidays for selected week and weather (if applies) using lambda expression

**Exit**
* Display exit 
* Prompt user if they are sure they want to exit
> * if yes, display 'Goodbye' and exit

### Holidays
We'll gather all the holidays and store in JSON.
* Make sure to represent holidays as objects in Python
* The List of Holidays should be represented as an object in Python
* The holiday class should include a way to display a holiday in the following format: `HolidayName (Date)`

In [1]:
from datetime import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
import inspect
from config import jsonlocation

In [2]:
##
@dataclass
class Holiday:
    name: str
    date: datetime
        
    def __str__ (self):
        # String output
        date_format = '%Y-%m-%d'
        date_str = datetime.strftime(self.date, date_format)
        # Holiday output when printed.
        return '%s (%s) ' % (self.name, date_str)

In [21]:
# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
class HolidayList:
    def __init__(self):
        self.innerHolidays = []
   
    def addHoliday(self, holidayObj):
        # Make sure holidayObj is a Holiday Object by checking the type
        if isinstance(holidayObj, Holiday):
            # Use innerHolidays.append(holidayObj) to add holiday
            self.innerHolidays.append(holidayObj)
            # print to the user that you added a holiday
            print(f'Success: {holidayObj} has been added to the holiday list.')
        else:
            print('Error: Please try again.')

    def findHoliday(self, HolidayName, Date):
        # Find Holiday in innerHolidays
        found_holiday = [x for x in self.innerHolidays if x.name == HolidayName and x.date == Date]
        # Return Holiday object
        try:
            found_hol_obj = found_holiday[0]
        except IndexError:
            found_hol_obj = None
        return found_hol_obj

    def removeHoliday(self, HolidayName, Date):
        # Holiday has already been found in innerList using the RemoveHoliday() function
        # remove the Holiday from innerHolidays
        self.innerHolidays = list(filter(lambda x : x.name != HolidayName and x.date != Date, self.innerHolidays))
        # inform user you deleted the holiday
        print(f'''
            Success:
            {HolidayName} has been removed from the list.
        ''')

    def read_json(self, jsonlocation):
        # Read in things from json file location
        with open(jsonlocation, 'r') as f:
            data = json.load(f)
            f.close()
        # Use addHoliday function to add holidays to inner list.
        for holiday in data['holidays']:
            holiday_name = holiday['name']
            holidate = holiday['date']
            date_format = '%Y-%m-%d'
            formatted_date = datetime.strptime(holidate, date_format).date()
            holidayobj = Holiday(holiday_name, formatted_date)
            self.addHoliday(holidayobj)

    def save_to_json(self, jsonlocation):
        #Makee empty holiday dictionary list
        holiday_dict_list= {"holidays" : []}
        #Gather individual holiday objects from innerHoliday list and append to holiday dict list
        for holiday in self.innerHolidays:
            holiday_name = holiday.name
            holidate = holiday.date
            date_format = '%Y-%m-%d'
            date = datetime.strftime(holidate, date_format)
            holiday_dict = {'name': holiday_name, 'date': date}
            holiday_dict_list['holidays'].append(holiday_dict)
        # Write out json file to selected file.
        with open(jsonlocation, 'w') as f:
            json.dump(holiday_dict_list, f, sort_keys = True, indent = 4)
            f.close()
        
    def scrapeHolidays():
        pass
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/ 
        # 2 previous years, current year, and 2  years into the future.
        # Scrape multiple years by adding year to the timeanddate URL e.g. https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.     

    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)
    
    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter innerHolidays by week number and year
        filtered_hol_list = list(
            filter(
                lambda x : x.date.isocalendar()[1] == week_number and x.date.isocalendar()[0] == year, self.innerHolidays
            )
        )
        # return your filtered holidays
        return filtered_hol_list

    def displayHolidaysInWeek(self, year, week_number):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        filt_hol_list = self.filter_holidays_by_week(year, week_number)
        # Output formated holidays in the week.
        for hol in filt_hol_list:
            print(hol)

    def getWeather(weekNum):
        pass
        # Convert weekNum to range between two days
        # Use Try / Except to catch problems
        # Query API for weather in that week range
        # Format weather information and return weather string.

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        current_year = datetime.now().isocalendar()[0]
        current_week = datetime.now().isocalendar()[1]
        # Use your filter_holidays_by_week function to get the list of holidays
        #current_week_hol = self.innerHolidays.filter_holidays_by_week(current_year, current_week)
        # Use your displayHolidaysInWeek function to display the holidays in the week
        self.displayHolidaysInWeek(current_year, current_week)
        # Ask user if they want to get the weather
        #get_weather = str(input('Would you like to see the weather? [y/n]: ')).lower()
        # If yes, use your getWeather function and display results
        #if get_weather == 'y':
        #    for hol in 

In [22]:
#UI Start Up
##
def StartUp(holiday_list):
    #Find number of holidays stored in system
    num_holidays = holiday_list.numHolidays()
    print(f'''
        Holiday Management
        =================================
        There are {num_holidays} holidays stored in the system
    ''')
    
    #MainMenu()

In [23]:
#UI MainMenu

def MainMenu(holiday_list):
    print(f'''
        Holiday Menu
        =====================================
    ''')
    #Store user selection from Main Menu
    user_selection = int(input('''
        1. Add a Holiday
        2. Remove a Holiday
        3. Save Holiday List
        4. View Holidays
        5. Exit
    '''))
    #Point to appropriate directory
    if user_selection == 1:
        AddHoliday(holiday_list)
    elif user_selection == 2:
        RemoveHoliday(holiday_list)
    elif user_selection == 3:
        SaveHolidayList(holiday_list)
#     elif user_selection == 4:
#         SaveChanges(name_of_tournament)
#     elif user_selection == 5:
#         Exit(name_of_tournament)


In [24]:
#UI Add Holiday
##
def AddHoliday(holiday_list):
    #Print welcome message
    print(f'''
        Add a Holiday
        ==============
    ''')
    #Gather Holiday name from user
    holiday_name = str(input("Holiday: "))
    #Gather Holiday date from user and ensure datetime
    incorrect_form = True
    while incorrect_form:
        try:
            testdate = input('Enter date (YYYY-MM-DD): ')
            date_format = '%Y-%m-%d'
            date = datetime.strptime(testdate, date_format).date()
        except ValueError:
            print('Invalid date. Please try again.')
        else:
            incorrect_form = False
            #Check if Holiday already exists in list
            found_holiday = holiday_list.findHoliday(holiday_name, date)
            #If exists, let user know
            if isinstance(found_holiday, Holiday):
                print(f'''
                    Error:
                    {holiday_name} already exists.
                ''')
            #If not found, addHoliday
            elif found_holiday == None:
                holiday_object = Holiday(holiday_name, date)
                holiday_list.addHoliday(holiday_object)
            
    #MainMenu()

In [25]:
#UI Remove Holiday
##
def RemoveHoliday(holiday_list):
    #Print welcome message
    print(f'''
        Remove a Holiday
        =================
    ''')
    holiday_not_in_list = True
    while holiday_not_in_list:
        #Gather holiday name from user
        holiday_name = str(input('Holiday Name: '))
        #Gather Holiday date from user and ensure datetime
        incorrect_form = True
        while incorrect_form:
            try:
                testdate = input('Enter date (YYYY-MM-DD): ')
                date_format = '%Y-%m-%d'
                date = datetime.strptime(testdate, date_format).date()
            except ValueError:
                print('Invalid date. Please try again.')
            else:
                incorrect_form = False
    
        #Check if holiday_name is in list
        found_holiday = holiday_list.findHoliday(holiday_name, date)
        if isinstance(found_holiday, Holiday):
            holiday_not_in_list = False
            holiday_list.removeHoliday(holiday_name, date)
        elif found_holiday == None:
            print(f'''
                Error:
                {holiday_name} not found.
            ''')
    
    
    #MainMenu()

In [26]:
#UI Save Holiday List
##
def SaveHolidayList(holiday_list):
    #Print welcome message
    print(f'''
        Save Holiday List
        ===================
    ''')
    #User save input
    user_save = str(input('Are you sure you want to save your changes? [y/n]: ')).lower()
    #Check user response
    if user_save == 'n':
        print('''
            Cancelled:
            Holiday list file save cancelled.
        ''')
    elif user_save == 'y':
        #Save list
        holiday_list.save_to_json(jsonlocation)
        print('''
            Success:
            Your Changes have been saved
        ''')
    
    
    #MainMenu()

In [27]:
#UI View Holidays

def ViewHolidays(holiday_list):
    #Print welcome message
    print(f'''
        View Holidays
        ==============
    ''')
    #User save input
    holiday_year = int(input('Which year?: '))
    #Ensure year is within [current year-2, current_year+2]
    holiday_week = input('Which week? #[1-52, Leave blank for the current week]: ')
    #Print holidays in week
    if holiday_week == '':
        print(f'These are the holidays for this week:')
        holiday_list.viewCurrentWeek()
        
    else:
        holiday_week = int(holiday_week)
        print(f'These are the holidays for {holiday_year} week#{holiday_week}:')
        holiday_list.displayHolidaysInWeek(holiday_year, holiday_week)
        
    
    #holiday_list.findHoliday(name, date)
    
    
    #MainMenu()

In [28]:
#Initiate holiday_list instance
a_holiday_list = HolidayList()

In [29]:
a_holiday_list.read_json(jsonlocation)

Success: Margaret Thatcher Day (2021-01-10)  has been added to the holiday list.
Success: World Sketchnote Day (2021-01-11)  has been added to the holiday list.
Success: Zanzibar Revolution Day (2021-01-12)  has been added to the holiday list.
Success: National Rubber Ducky Day (2021-01-13)  has been added to the holiday list.
Success: Tamil Thai Pongdal Day (2021-01-14)  has been added to the holiday list.
Success: National Bagel Day (2021-01-15)  has been added to the holiday list.
Success: Signing of the Peace Accords (2021-01-16)  has been added to the holiday list.
Success: Christmas Day (2022-12-25)  has been added to the holiday list.


In [30]:
#Add holiday to holiday_list
AddHoliday(a_holiday_list)


        Add a Holiday
    
Holiday: Christian Day
Enter date (YYYY-MM-DD): 2022-07-10
Success: Christian Day (2022-07-10)  has been added to the holiday list.


In [14]:
print(a_holiday_list.innerHolidays)

[Holiday(name='Margaret Thatcher Day', date=datetime.date(2021, 1, 10)), Holiday(name='World Sketchnote Day', date=datetime.date(2021, 1, 11)), Holiday(name='Zanzibar Revolution Day', date=datetime.date(2021, 1, 12)), Holiday(name='National Rubber Ducky Day', date=datetime.date(2021, 1, 13)), Holiday(name='Tamil Thai Pongdal Day', date=datetime.date(2021, 1, 14)), Holiday(name='National Bagel Day', date=datetime.date(2021, 1, 15)), Holiday(name='Signing of the Peace Accords', date=datetime.date(2021, 1, 16)), Holiday(name='Christmas Day', date=datetime.date(2022, 12, 25)), Holiday(name="New Year's Day", date=datetime.date(2022, 1, 1)), Holiday(name='Christian Day', date=datetime.date(2022, 7, 10))]


In [32]:
ViewHolidays(a_holiday_list)


        View Holidays
    
Which year?: 2021
Which week? #[1-52, Leave blank for the current week]: 2
These are the holidays for 2021 week#2:
World Sketchnote Day (2021-01-11) 
Zanzibar Revolution Day (2021-01-12) 
National Rubber Ducky Day (2021-01-13) 
Tamil Thai Pongdal Day (2021-01-14) 
National Bagel Day (2021-01-15) 
Signing of the Peace Accords (2021-01-16) 


In [12]:
a_holiday_list.filter_holidays_by_week(2021, 2)

[Holiday(name='World Sketchnote Day', date=datetime.date(2021, 1, 11)),
 Holiday(name='Zanzibar Revolution Day', date=datetime.date(2021, 1, 12)),
 Holiday(name='National Rubber Ducky Day', date=datetime.date(2021, 1, 13)),
 Holiday(name='Tamil Thai Pongdal Day', date=datetime.date(2021, 1, 14)),
 Holiday(name='National Bagel Day', date=datetime.date(2021, 1, 15)),
 Holiday(name='Signing of the Peace Accords', date=datetime.date(2021, 1, 16))]

In [11]:
#Create a dict from HolidayList obj

holiday_dict_list= {"holidays" : []}
#print(type(holiday_dict_list['holidays']))
#print(a_holiday_list.innerHolidays[0])

for holiday in a_holiday_list.innerHolidays:
    holiday_name = holiday.name
    holidate = holiday.date
    date_format = '%Y-%m-%d'
    date = datetime.strftime(holidate, date_format)
    holiday_dict = {'name': holiday_name, 'date': date}
    holiday_dict_list['holidays'].append(holiday_dict)
    #holidayobj = Holiday(holiday_name, date)
    #a_holiday_list.addHoliday(holidayobj)
    

#json_test = json.dumps(a_holiday_list.innerHolidays)
#print(json_test)

In [17]:
first_in_list = data['holidays'][0]
unformatted_date = first_in_list['date']
date_format = '%Y-%m-%d'
date = datetime.strptime(unformatted_date, date_format).date()
print(date)

2021-01-10


In [14]:
json_test = json.dumps(holiday_dict_list)
print(json_test)

{"holidays": [{"name": "Margaret Thatcher Day", "date": "2021-01-10"}, {"name": "World Sketchnote Day", "date": "2021-01-11"}, {"name": "Zanzibar Revolution Day", "date": "2021-01-12"}, {"name": "National Rubber Ducky Day", "date": "2021-01-13"}, {"name": "Tamil Thai Pongdal Day", "date": "2021-01-14"}, {"name": "National Bagel Day", "date": "2021-01-15"}, {"name": "Signing of the Peace Accords", "date": "2021-01-16"}, {"name": "Christmas Day", "date": "2022-12-25"}]}


In [58]:
#Remove holiday from holiday list
RemoveHoliday(a_holiday_list)


        Remove a Holiday
    
Holiday Name: St Patties Day
Enter date (YYYY-MM-DD): 2022-03-17

                Error:
                St Patties Day not found.
            
Holiday Name: New Year's Day
Enter date (YYYY-MM-DD): 2022-01-01

            Success:
            New Year's Day has been removed from the list.
        


Christmas Day (2022-12-25) 


In [42]:
print(a_holiday_list.innerHolidays)

[Holiday(name='Margaret Thatcher Day', date=datetime.date(2021, 1, 10)), Holiday(name='World Sketchnote Day', date=datetime.date(2021, 1, 11)), Holiday(name='Zanzibar Revolution Day', date=datetime.date(2021, 1, 12)), Holiday(name='National Rubber Ducky Day', date=datetime.date(2021, 1, 13)), Holiday(name='Tamil Thai Pongdal Day', date=datetime.date(2021, 1, 14)), Holiday(name='National Bagel Day', date=datetime.date(2021, 1, 15)), Holiday(name='Signing of the Peace Accords', date=datetime.date(2021, 1, 16)), Holiday(name='Christmas Day', date=datetime.date(2022, 12, 25)), Holiday(name="New Year's Day", date=datetime.date(2022, 1, 1))]


In [24]:
AddHoliday(a_holiday_list)


        Add a Holiday
    
Holiday: Christmas Day
Enter date (YYYY-MM-DD): 2022-12-25

                    Error:
                    Christmas Day already exists.
                


In [17]:
[x for x in a_holiday_list.innerHolidays if x.name == "Christmas Day" and x.date == datetime(2022, 12, 25).date()]

[Holiday(name='Christmas Day', date=datetime.date(2022, 12, 25))]

In [ ]:

def main():
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    holiday_list = HolidayList()
    # 2. Load JSON file via HolidayList read_json function
    holiday_list.read_json(jsonlocation)
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going. 



In [11]:
SaveHolidayList(a_holiday_list)


        Save Holiday List
    
Are you sure you want to save your changes? [y/n]: y

            Success:
            Your Changes have been saved
        


In [13]:
holiday_week = input('Which week? #[1-52, Leave blank for the current week]: ')

Which week? #[1-52, Leave blank for the current week]: 


In [14]:
type(holiday_week)

str

In [ ]:
#UI Exit

def Exit():
    #Print welcome message
    print(f'''
        Exit
        =====
    ''')
    #User exit input
    if changes_to_be_saved == True:
        user_exit = str(input('Are you sure you want to exit? [y/n]: ')).lower()
    elif changes_to_be_saved == False:
        user_exit = str(input('''
            Are you sure you want to exit?
            Your changes will be lost.
            [y/n]: 
            ''')).lower()
    
    if user_exit == 'y':
        break
    elif user_exit == 'n':
        MainMenu()
    